In [1]:
import datetime as dt
import requests
import random
import time
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
# URL to the data extraction site
url = "https://metsonline.dosm.gov.my/tradev2/product-coderesult"

# Obtain csrf_token from the website
csrf_token = 'dG5PZ09oay0nPiIMATBTeTsZHFAgKSJdRQQVKwRYJBQADRckHStGYA=='

# Create headers for all posts
headers = {
"Content-Type": "application/x-www-form-urlencoded; charset=UTF-8", 
"Cookie": "_ga_F8QG3XBL63=GS1.1.1679998862.1.1.1679998930.0.0.0; _ga_7Q47WMYPD5=GS1.1.1679998865.1.1.1679998930.0.0.0; _ga_HK905W412V=GS1.1.1679998865.1.1.1679998930.0.0.0; mp_f55a9b6a137ab21675ef8724f7864bde_mixpanel=%7B%22distinct_id%22%3A%20%2218727cb0e099da-0247f50f77e9aa-7a545471-144000-18727cb0e0a31%22%2C%22%24device_id%22%3A%20%2218727cb0e099da-0247f50f77e9aa-7a545471-144000-18727cb0e0a31%22%2C%22%24search_engine%22%3A%20%22google%22%2C%22%24initial_referrer%22%3A%20%22https%3A%2F%2Fwww.google.com%2F%22%2C%22%24initial_referring_domain%22%3A%20%22www.google.com%22%2C%22__timers%22%3A%20%7B%7D%7D; _ga_TF34V4E8EM=GS1.1.1679999831.1.1.1680004133.0.0.0; _ga_XSB7QCQ0M9=GS1.1.1681551362.3.1.1681551368.0.0.0; _ga=GA1.1.324035783.1679998837; PHPSESSID=3oi1at9dfj4ugn6nno2bsqdd60; _csrf=193a3c282709e8f8d32929d94372b30d2cc2a5402306edc5e12c04ca098a1819a%3A2%3A%7Bi%3A0%3Bs%3A5%3A%22_csrf%22%3Bi%3A1%3Bs%3A32%3A%22SPmkNX8TOwS7oAIp1jZLK0O9tcXCRC-M%22%3B%7D; BNES_PHPSESSID=fk0GuBGEQJmmIMfAn9TJ6EK+FW/oxjgTr2AAO+vNTw1JHh1+YzlOkigmNmbMOtjxuyDVXvBqWx9L6ZHOVdyK0Unb9r8ZsuhgKRw0f3zlmA8=; BNES__csrf=T1JRVt/fspwmarDUSua9ANdq7/OTqhlhu8lprfzaWWaQtpaDzljVNyJUL1Y65guWawDI8LDmCHrD6nlIXud7o3ytcCBG5TDvi4Kd94SBmSCx1ED1HWM0YV+ztPXAQ5AhjCkcJExisSE5tRQSObfAv0WAFW/4frqxEg4pTyBtkcJZTi4Amki6jSChfqJAyShFuYOrg/ZKI0IsB8ShUR5UFcxqa3a28w/OAICYVhchalmoAAhmCMY22tozvta3gpBJGC+pONVCkNPozWC+7XVzDC5ZciTE34Uq"
}

In [3]:
# To obtain the targeted year with selected interval
start_year = 2000
current_year = dt.date.today().year
interval_year = 4

targeted_year = []

for year in range(start_year, current_year + 1, interval_year):
    if year + interval_year < 2023:
        ending_year = year + interval_year - 1
        targeted_year.append(ending_year)
    else:
        ending_year = current_year
        targeted_year.append(ending_year)
        
print(targeted_year)

[2003, 2007, 2011, 2015, 2019, 2023]


In [4]:
# To start extract data from url via request
data_raw = []
start = start_year

max_retries = 2

for year in targeted_year:
    payload_month = {
    '_csrf': csrf_token,
    'Tradev2[typeofsearch]': 'classification',
    'Tradev2[typedigit]': 7,
    'Tradev2[rangecode1]': 0,
    'Tradev2[rangecode2]': 9,
    # 'Tradev2[code_idcode]': ,
    # 'Tradev2[code_idcodedigit9]': ,
    # 'Tradev2[tradeflow]': ,
    'Tradev2[tradeflow][]': 'exports',
    # 'Tradev2[timeframe]': ,
    'Tradev2[timeframe]': 'month',
    # 'Tradev2[rangeyear]': ,
    # 'Tradev2[rangeyear2]': ,
    # 'Tradev2[rangeyearone]': ,
    # 'Tradev2[rangemonthone]': ,
    'Tradev2[mothdata]': start,
    'Tradev2[mothdata2]': year,
    # 'Tradev2[classification_serch]': ,
    # 'Tradev2[country2]': ,
    'Tradev2[geogroup]': 1,
    'Tradev2[geogroup]': 29,
    'Tradev2[codeshowby]': 'code'
    }
    
    for retry in range(max_retries):
        retry_delay = random.randint(10,15)  # seconds
        try:
            # Process the response here
            monthly_raw = requests.post(url, data=payload_month, headers=headers)
            data_raw.append(monthly_raw) # Store the extracted raw data into a list
            print(f"Data Extraction for {start}-{year} is Successful.")
            time.sleep(5)
            break # Break the retry loop if successful
            
        except (requests.ConnectionError) as e:
            print(f"Error: {e}. Retrying in {retry_delay} seconds...")
            time.sleep(retry_delay)
            
        except Exception as e:
            print(f"An error occurred: {e}")
            break  # Stop retrying if a different error occurs
    
    if year < 2023:
        start = year + 1
     
print(data_raw)

Data Extraction for 2000-2003 is Successful.
Data Extraction for 2004-2007 is Successful.
Data Extraction for 2008-2011 is Successful.
Data Extraction for 2012-2015 is Successful.
Data Extraction for 2016-2019 is Successful.
Data Extraction for 2020-2023 is Successful.
[<Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>]


In [12]:
# Create an empty list for storing the raw text from data extraction
data_extracted = []

# To extract only the table value
for raw in data_raw:
    result = BeautifulSoup(raw.text, 'html.parser') # Parse the HTML
    table = result.find('table', class_='table-bordered') # Look up for the table
    
    # Extract table rows
    rows = table.find_all('tr')
    for row in rows:
        cols = row.find_all(['th', 'td'])
        cols = [col.get_text(strip=True) for col in cols]
        data_extracted.append(cols)